# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [2]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [33]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks.describe()

,Ticker
count,501
unique,501
top,WELL
freq,1


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [4]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [5]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data


{'avgTotalVolume': 93959043,
 'calculationPrice': 'close',
 'change': 1.5,
 'changePercent': 0.01177,
 'close': 133.7,
 'closeSource': 'flioifca',
 'closeTime': 1657470554231,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': 134.89,
 'delayedPriceTime': 1681858251534,
 'extendedChange': -0.01,
 'extendedChangePercent': -8e-05,
 'extendedPrice': 135.09,
 'extendedPriceTime': 1714052661577,
 'high': 134.533,
 'highSource': 'l eipryedetcuieanm1d 5 ',
 'highTime': 1732929621687,
 'iexAskPrice': None,
 'iexAskSize': None,
 'iexBidPrice': None,
 'iexBidSize': None,
 'iexClose': 132.98,
 'iexCloseTime': 1692484413794,
 'iexLastUpdated': None,
 'iexMarketPercent': None,
 'iexOpen': 135.12,
 'iexOpenTime': 1697932140230,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexVolume': None,
 'lastTradeTime': 1734413847425,
 'latestPrice': 136.22,
 'latestSource': 'Close',
 'latestTime': 'June 17, 2022',
 'latestUpdate': 1656329355729,
 'latestVolume': 138676331,
 'low': 133.1

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [6]:
price = data['latestPrice']
market_cap = data['marketCap']
print(price, market_cap)

136.22 2154733309273


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [7]:
my_columns = [
    'Ticker',
    'Stock Price',
    'Market Capitalization',
    'Number of Shares to Buy',
]
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [8]:
final_dataframe.append(
    pd.Series(
    [
        symbol,
        price,
        market_cap,
        'N/A',
    ],
    index = my_columns
    ),
    ignore_index = True
)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,136.22,2154733309273,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [9]:
final_dataframe = pd.DataFrame(columns = my_columns)

for stock in stocks['Ticker'][:5]:
    symbol = stock
    api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
        pd.Series(
        [
            symbol,
            data['latestPrice'],
            data['marketCap'],
            'N/A',
        ],
        index = my_columns
        ),
        ignore_index = True
    )
    

In [10]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,116.76,34452497828,N/A
1,AAL,13.14,8665287378,N/A
2,AAP,174.17,10551171379,N/A
3,AAPL,132.85,2186158307416,N/A
4,ABBV,144.98,249085423569,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [11]:
def chunks(list, n):
    for i in range(0, len(list), n):
            yield list[i:i +n]

In [34]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['marketCap'],
                'N/A',
            ],
            index = my_columns
            ),
            ignore_index = True
        )

final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,113.63,34194049378,N/A
1,AAL,13.32,8703582347,N/A
2,AAP,169.68,10268295332,N/A
3,AAPL,132.98,2188857701528,N/A
4,ABBV,143.66,246227517375,N/A
...,...,...,...,...
496,YUM,113.50,31276230450,N/A
497,ZBH,103.13,21998388462,N/A
498,ZBRA,293.46,15588620551,N/A
499,ZION,54.10,8165043452,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [67]:
portfolio_size = input('Enter the value of your portfolio:')

try:
    portfolio_size = float(portfolio_size)
except ValueError:
    print('That is not an integer!\nPlease try again')
    portfolio_size = input('Enter the value of your portfolio:')
    portfolio_size = float(portfolio_size)


Enter the value of your portfolio:10000000


In [68]:
position_size = portfolio_size / len(final_dataframe.index)
for i in range(len(final_dataframe.index)):
#     print(final_dataframe.loc[i, 'Number of Shares to Buy'])
    final_dataframe.iloc[i, 3] = math.floor(position_size / final_dataframe.iloc[i, 1])

final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,113.63,34194049378,175
1,AAL,13.32,8703582347,1498
2,AAP,169.68,10268295332,117
3,AAPL,132.98,2188857701528,150
4,ABBV,143.66,246227517375,138
...,...,...,...,...
496,YUM,113.50,31276230450,175
497,ZBH,103.13,21998388462,193
498,ZBRA,293.46,15588620551,68
499,ZION,54.10,8165043452,368


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [111]:
writer = pd.ExcelWriter('recommended trade.xlsx', engine = 'xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trade', index = False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [112]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1,
    }
)
dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1,
    }
)
integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1,
    }
)

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [113]:
# writer.sheets['Recommended Trade'].set_column('A:A', 18, string_format)
# writer.sheets['Recommended Trade'].set_column('B:B', 18, string_format)
# writer.sheets['Recommended Trade'].set_column('C:C', 18, string_format)
# writer.sheets['Recommended Trade'].set_column('D:D', 18, string_format)

# writer.sheets['Recommended Trade'].write('A1', 'Ticker', string_format)
# writer.sheets['Recommended Trade'].write('B1', 'Ticker', string_format)
# writer.sheets['Recommended Trade'].write('C1', 'Ticker', string_format)
# writer.sheets['Recommended Trade'].write('D1', 'Ticker', string_format)

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [114]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', integer_format],
}

for column in column_formats.keys():
    writer.sheets['Recommended Trade'].set_column(f'{column}:{column}', len(column_formats[column][0]), column_formats[column][1])
    writer.sheets['Recommended Trade'].write(f'{column}1', column_formats[column][0], string_format)

## Saving Our Excel Output

Saving our Excel file is very easy:

In [115]:
writer.save()